# Deploying NVIDIA Nemotron-3-Nano with TensorRT LLM

This notebook will walk you through how to run the `nnvidia/NVIDIA-Nemotron-3-Nano-30B-A3B` model via TensorRT-LLM

[TensorRT LLM](https://nvidia.github.io/TensorRT-LLM/) is NVIDIA’s open-source library for accelerating and optimizing LLM inference performance on NVIDIA GPUs. TRTLLM support for this model is enabled through the AutoDeploy workflow. More details about this workflow can be found [here](https://nvidia.github.io/TensorRT-LLM/features/auto_deploy/auto-deploy.html).

For more details on the model [click here](https://huggingface.co/nvidia/NVIDIA-Nemotron-3-Nano-30B-A3B-FP8). 

Prerequisites:
- NVIDIA GPU with recent drivers (≥ 60 GB VRAM for BF16, ≥ 32 GB for FP8) and CUDA 12.x
- Python 3.10+
- TensorRT-LLM (you can refer to NVIDIA documentation, or pull this [container](https://catalog.ngc.nvidia.com/orgs/nvidia/teams/tensorrt-llm/containers/release?version=1.2.0rc5))

#### Launch on NVIDIA Brev
You can simplify the environment setup by using [NVIDIA Brev](https://developer.nvidia.com/brev). Click the button to launch this project on a Brev instance with the necessary dependencies pre-configured.

Once deployed, click on the "Open Notebook" button to get started with this guide. 

[![Launch on Brev](https://brev-assets.s3.us-west-1.amazonaws.com/nv-lb-dark.svg)](https://brev.nvidia.com/launchable/deploy?launchableID=env-36ikYKeRmXqG8MJjxsgROJM4S2V)

## Prerequisites & environment

Set up a containerized environment for TensorRT-LLM by running the following command in a terminal.

```shell
docker run --rm -it --ipc=host --ulimit memlock=-1 --ulimit stack=67108864 --gpus=all -p 8000:8000 nvcr.io/nvidia/tensorrt-llm/release:1.2.0rc5
```

You now have TRT-LLM set up! 

In [1]:
#If pip not found
!python -m ensurepip --default-pip

Looking in links: /tmp/tmpuvjbo8y1


In [ ]:
%pip install torch openai

## Verify GPU

Check that CUDA is available and the GPU is detected correctly.


In [3]:
# Environment check
import sys
import torch

print(f"Python: {sys.version}")
print(f"CUDA available: {torch.cuda.is_available()}")
print(f"Num GPUs: {torch.cuda.device_count()}")

if torch.cuda.is_available():
    for i in range(torch.cuda.device_count()):
        print(f"GPU[{i}]: {torch.cuda.get_device_name(i)}")


Python: 3.12.12 (main, Dec  9 2025, 19:02:36) [Clang 21.1.4 ]
CUDA available: True
Num GPUs: 1
GPU[0]: NVIDIA H100 80GB HBM3


## OpenAI-compatible server

Start a local OpenAI-compatible server with TensorRT-LLM via the terminal, within the running docker container.

Ensure that the following commands are executed from the docker terminal.

#### Create a YAML file with the required configuration

```shell
cat > nano_v3.yaml<<EOF
runtime: trtllm
compile_backend: torch-cudagraph
max_batch_size: 64
max_seq_len: 16384
enable_chunked_prefill: true
attn_backend: flashinfer
model_factory: AutoModelForCausalLM
skip_loading_weights: false
free_mem_ratio: 0.65
cuda_graph_batch_sizes: [1, 2, 4, 8, 16, 24, 32, 64, 128, 256, 320, 384]
kv_cache_config:
  # disable kv_cache reuse since not supported for hybrid/ssm models
  enable_block_reuse: false
transforms:
  detect_sharding:
    sharding_dims: ['ep', 'bmm']
    allreduce_strategy: 'AUTO'
    manual_config:
      head_dim: 128
      tp_plan:
        # mamba SSM layer
        "in_proj": "mamba"
        "out_proj": "rowwise"
        # attention layer
        "q_proj": "colwise"
        "k_proj": "colwise"
        "v_proj": "colwise"
        "o_proj": "rowwise"
        # NOTE: consider not sharding shared experts and/or
        # latent projections at all, keeping them replicated.
        # To do so, comment out the corresponding entries.
        # moe layer: SHARED experts
        "up_proj": "colwise"
        "down_proj": "rowwise"
        # MoLE: latent projections: simple shard
        "fc1_latent_proj": "gather"
        "fc2_latent_proj": "gather"
  multi_stream_moe:
    stage: compile
    enabled: true
  insert_cached_ssm_attention:
      cache_config:
        mamba_dtype: float32
  fuse_mamba_a_log:
    stage: post_load_fusion
    enabled: true
EOF
```


#### Load the model

##### BF16 version

```shell
TRTLLM_ENABLE_PDL=1 trtllm-serve "nvidia/NVIDIA-Nemotron-3-Nano-30B-A3B-BF16" \
--host 0.0.0.0 \
--port 8000 \
--backend _autodeploy \
--trust_remote_code \
--reasoning_parser deepseek-r1 \
--tool_parser qwen3_coder \
--extra_llm_api_options nano_v3.yaml
```

##### Alternative: Load the FP8 quantized version for faster inference and lower memory usage

```shell
TRTLLM_ENABLE_PDL=1 trtllm-serve "nvidia/NVIDIA-Nemotron-3-Nano-30B-A3B-FP8" \
--host 0.0.0.0 \
--port 8000 \
--backend _autodeploy \
--trust_remote_code \
--reasoning_parser deepseek-r1 \
--tool_parser qwen3_coder \
--extra_llm_api_options nano_v3.yaml
```

Your server is now running!

#### Use the API

Use the OpenAI-compatible client to send requests to the TensorRT-LLM server.

Note: The model supports two modes - Reasoning ON (default) vs OFF. This can be toggled by setting enable_thinking to False, as shown below. 

In [4]:
from openai import OpenAI
import requests

# Setup client
BASE_URL = "http://0.0.0.0:8000/v1"
API_KEY = "null" 
client = OpenAI(base_url=BASE_URL, api_key=API_KEY)

model_id = "nvidia/NVIDIA-Nemotron-3-Nano-30B-A3B-BF16" # set this to the model you loaded

In [5]:
# Reasoning on (default)
print("Reasoning on")
response = client.chat.completions.create(
    model=model_id,
    messages=[
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": "Give me 3 bullet points about TensorRT-LLM."}
    ],
    temperature=1,
    max_tokens=256,
)
print(response.choices[0].message.reasoning_content, response.choices[0].message.content)

print("\n")

# Reasoning off
print("Reasoning off")
response = client.chat.completions.create(
    model=model_id,
    messages=[
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": "Give me 3 bullet points about TensorRT-LLM."}
    ],
    temperature=0,
    max_tokens=256,
    extra_body={"chat_template_kwargs": {"enable_thinking": False}}
)
print(response.choices[0].message.reasoning_content, response.choices[0].message.content)

Reasoning on
We need to respond with 3 bullet points about TensorRT-LLM. Provide concise, factual points. Should we add any extra explanation? They just asked for 3 bullet points. We'll produce exactly 3 bullet points.
 
- **High‑performance inference**: TensorRT‑LLM leverages NVIDIA’s TensorRT optimizations (kernel fusion, INT8/FP8 quantization, and graph-level transformations) to accelerate large‑language‑model generation on GPUs, delivering up to 10× lower latency and higher throughput compared to naïve implementations.  

ourgward’s LLM inference engine built on NVIDIA’s **TensorRT-LLM** library efficiently maps transformer operators to Tensor Core kernels, enabling mixed‑precision (BF16/FP8) execution while preserving model accuracy.  

- **Scalable across models and hardware**: Supports a wide range of architectures (GPT‑J, LLaMA, Falcon, etc.) and GPU families (A100, H100, RT participou) with dynamic tensor shapes and automatic batching, allowing deployment from edge devices to 

In [6]:
# Streaming chat completion
print("Streaming response:")
stream = client.chat.completions.create(
    model=model_id,
    messages=[
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": "What are the first 5 prime numbers?"}
    ],
    temperature=0.7,
    max_tokens=1024,
    stream=True,
)

for chunk in stream:
    if chunk.choices[0].delta.content:
        print(chunk.choices[0].delta.content, end="", flush=True)

Streaming response:

The first 5 prime numbers are:  
**2, 3, 5, 7, 11**

### Why?
- **Prime numbers**

### Tool calling

Use the OpenAI tools schema to call functions via the TensorRT-LLM endpoint.

In [9]:
# Tool calling via OpenAI tools schema
TOOLS = [
    {
        "type": "function",
        "function": {
            "name": "calculate_tip",
            "parameters": {
                "type": "object",
                "properties": {
                    "bill_total": {
                        "type": "integer",
                        "description": "The total amount of the bill"
                    },
                    "tip_percentage": {
                        "type": "integer",
                        "description": "The percentage of tip to be applied"
                    }
                },
                "required": ["bill_total", "tip_percentage"]
            }
        }
    }
]

completion = client.chat.completions.create(
    model=model_id,
    messages=[
        {"role": "system", "content": ""},
        {"role": "user", "content": "My bill is $50. What will be the amount for 15% tip?"}
    ],
    tools=TOOLS,
    temperature=0.6,
    top_p=0.95,
    max_tokens=512,
    stream=False
)

print(completion.choices[0].message.reasoning_content, completion.choices[0].message.content)
print(completion.choices[0].message.tool_calls)

Okay, the user wants to calculate a 15% tip on a $50 bill. Let me check the tools available. There's a function called calculate_tip that takes bill_total and tip_percentage. The parameters are integers, so I need to convert 15% to 15 and $50 to 50. I'll call the function with these values. The function should return the tip amount, which I can then present to the user.
 


[ChatCompletionMessageFunctionToolCall(id='chatcmpl-tool-a516606befc240c893817b3480c0db5b', function=Function(arguments='{"bill_total": 50, "tip_percentage": 15}', name='calculate_tip'), type='function')]
